In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt

import numpy as np

from deep_hedging import (
    Underlyings,
    Ticker,
    ConstantRateCurve,
    EuropeanCall,
    EuropeanPut,
)
from deep_hedging.monte_carlo import GBMPricer

RANDOM_SEED = 12

In [3]:
N_PATHS = 100_000
N_STOCKS = 1
TILL_MATURITY = 3.0
N_DAYS = 365 * TILL_MATURITY
RF_RATE = 0.03
VOL = 0.2

In [4]:
N_DAYS

1095.0

In [5]:
np.random.seed(RANDOM_SEED)

start = dt.datetime(2019, 1, 1)
end = start + dt.timedelta(days=N_DAYS)

underlyings = Underlyings(
    tickers=[Ticker(f"Stock {i + 1}", f"{i + 1}") for i in range(N_STOCKS)],
    start=start,
    end=end,
    dividends=np.array([0.0] * N_STOCKS),
    means=np.array([RF_RATE] * N_STOCKS),
    var_covar=np.array([VOL**2] * N_STOCKS),
)
underlyings.data

In [6]:
np.sqrt(np.diag(underlyings.get_var_covar()))

array([[0.2]])

In [7]:
curve = ConstantRateCurve(constant_rate=RF_RATE)

In [8]:
european_call = EuropeanCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=1.0,
    start_date=start,
    end_date=end,
)

In [9]:
pricer = GBMPricer(payoff_function=european_call.payoff, random_seed=RANDOM_SEED)

In [10]:
pricer.price(
    current_spot=[1.0],
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_rate,
    dividends_fn=lambda t: 0.0,
    var_covar_fn=lambda t: np.array([VOL**2]),
    n_paths=N_PATHS,
)

0.17888212769640016

In [11]:
european_call.price()

array([[0.17899527]])